In [ ]:
from openeye.oechem import OEGraphMol, OESmilesToMol
def smi_to_graphmol(smi):
    mol = OEGraphMol()
    stat = OESmilesToMol(mol, smi)
    return mol if stat else False

def prep_mol(mol):
    return OECenter(mol)

def apply_to_mol(mol, funcs, verbose=True):
# # TODO: ??? Allow assignmnet where asgn = {'some name': lambda x: x} mol.SetData(asgn.keys(i)) = asgn(mol)
# # funcs --> https://docs.eyesopen.com/toolkits/python/oechemtk/OEChemFunctions/index.html
    for fxn in funcs:
        err = 'Error applying function %s to %s' % (fxn.__name__, mol.__name__)
        if verbose and not fxn(mol): print(err) 
    return None

    

In [ ]:
def min_grid_volume(mols):
    max_coords = (0,0,0)
    for mol in mols:
        for atom in mol.GetAtoms():
            coords = mol.GetCoords(atom)
            for i, x in enumerate(max_idx):
                if coords[i] > x:
                    max_coords[i] = coords[i]
    return max_coords

def make_min_grid(mols, grid, padding=[0,0,0]):
# Create an empty grid with ths smallest possible x,y,z dimensions large enough to contain every (centered) molecule in mols. Then, add the specified padding
    min_x,min_y,min_z = min_grid_volume(mols)
    return grid(min_x,min_y,min_z)

def make_grid_cext():
# # Serves the same purpose as make_min_grid()
#     OECenter(mol)
#     someextens [OEGetCenterAndExtents(mol) for mol in mols]
#     OEMakeGridFromCenterAndExtents(max(somextents))
    pass

In [ ]:
# # Workflow wherein the channel list IS NOT passed to the embedding function
# embeddings = [
#     {
#         grid: OEScalarGrid(gargs) # Or make_min_grid((x,y,z), center, padding)
#         func: lambda atom, mol, grid: atom.GetPartialCharge()
#     },
#     {
#         grid: OEScalarGrid(gargs) # Or make_min_grid((x,y,z), center, padding)
#         func: lambda atom, mol, grid: OEMakeMolecularGaussianGrid(grid, mol, 0.5)
#     },
#     ...
# ]

def map_grid(grid, mol, func=lambda atom, mol, grid: atom.GetPartialCharge()):
    for atom in mol.GetAtoms():
        x,y,z = mol.GetCoords(atom)
        if grid.IsInGrid(x, y, z):
            grid.SetValue(x, y, z, func(atom, mol, grid))
    return grid

def embed_grid(mols, embeddings):
    cols = list()
    for mol in mols:
        channels = []
        for embedding in channel_embeddings:
            grid = embedding.grid
            func = embedding.func
            ebdg = map_grid(grid, mol, func)
            channels.append(ebdg)
        mol.SetData('channels', channels)
        cols.append(mol)
    return cols

In [ ]:
# # Workflow wherein the channel list IS passed to the embedding func
# channel_embeddings = [
#     {
#         grid: OEScalarGrid(gargs) # Or make_min_grid((x,y,z), center, padding)
#         func: lambda atom, mol, grid, chan: atom.GetPartialCharge()
#     },
#     ...
# ]

def map_channels(grid, mol, funcs=[lambda atom, mol, grid, chans: atom.GetPartialCharge()]):
    # Create N = len(funcs) channels for a single mol via:
    # # return --> channels = [grid1,grid2,grid3,...].shape(N) where each grid is an embedding of mol calculated by applying to mol the corresponding atom descriptor for that channel. 
    chans = []
    for fxn in funcs:
        for atom in mol.GetAtoms():
            x,y,z = mol.GetCoords(atom)
            if grid.IsInGrid(x, y, z):
                grid.SetValue(x, y, z, fxn(atom, mol, grid, chans))
    return chans

def embed_channels(mols, embeddings):
    pass